In [ ]:
#default_exp docker_startup

# Docker Helpers

> An intuitive CLI to help ease your Docker setup

## What are these helpers?

These include helpers for you to understand your docker images quickly and efficiently.

In [ ]:
#hide
base = """nbdev

docker run --expose 8888/tcp -l repo2docker.ref='None' -l repo2docker.repo='local' -l repo2docker.version='2021.08.0' -e 'PATH=/home/runner/.local/bin:/home/runner/.local/bin:/srv/conda/envs/notebook/bin:/srv/conda/bin:/srv/npm/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin' -e 'DEBIAN_FRONTEND=noninteractive' -e 'LC_ALL=en_US.UTF-8' -e 'LANG=en_US.UTF-8' -e 'LANGUAGE=en_US.UTF-8' -e 'SHELL=/bin/bash' -e 'USER=runner' -e 'HOME=/home/runner' -e 'APP_BASE=/srv' -e 'NPM_DIR=/srv/npm' -e 'NPM_CONFIG_GLOBALCONFIG=/srv/npm/npmrc' -e 'CONDA_DIR=/srv/conda' -e 'NB_PYTHON_PREFIX=/srv/conda/envs/notebook' -e 'NB_ENVIRONMENT_FILE=/tmp/env/environment.lock' -e 'KERNEL_PYTHON_PREFIX=/srv/conda/envs/notebook' -e 'REPO_DIR=/home/runner' -e 'CONDA_DEFAULT_ENV=/srv/conda/envs/notebook' -e 'PYTHONUNBUFFERED=1' -d --entrypoint "/usr/local/bin/repo2docker-entrypoint" sha256:55047ce356e618ce8cc50f9de0cd28f0c1c808665464f738854b2c939dca2c5d 'jupyter' 'notebook' '--ip' '0.0.0.0'"""

In [ ]:
#export
from rich import print as rprint
from rich.table import Table
from docker_buddy.cli_colors import _run, console
from fastcore.script import call_parse, Param

In [ ]:
#export
def _parse_arguments(args:str):
    "Seperates out additional arguments for an image, and grabs their default values"
    sets = []
    is_pair = False
    curr_pair = []
    for word in args:
        if word.startswith('-'):
            if is_pair:
                curr_pair[0] += f', {word}'
            else:
                is_pair = True
                curr_pair.append(word)
            continue
        elif is_pair:
            curr_pair.append(word)
            is_pair = False
            sets.append(curr_pair)
            curr_pair = []
    return sets

In [ ]:
#export
def _format_inspection(inspection:str):
    "Formats prettily an inspection from rekcod for an image"
    _color_a, _color_b, _color_c = "#0C7BDC", "#40B0A6", "#0C7BDC"
    image_name, _, command = inspection.split('\n')
    start_cmd = command.split('-')[0]
    s = f'[bold {_color_a}]Image[/bold {_color_a}]: {image_name}\n\n[bold {_color_a}]Command for Running Container[/bold {_color_a}]: {start_cmd}\n\n'
    s += f'[bold {_color_a}]Possible Arguments:[/bold {_color_a}]'
    table = Table(show_header=True, header_style=f'bold {_color_a}', show_lines=True)
    table.add_column('Argument(s)', justify="center")
    table.add_column('Value', justify='Center')
    sets = _parse_arguments(command.split(' '))
    for arg, val in sets:
        if ',' in arg:
            def _add_color(o): return f'[{_color_b}]{o}[/{_color_b}]'
            arg = ','.join([_add_color(o) for o in arg.split(',')])
        else:
            arg = f'[{_color_b}]{arg}[/{_color_b}]'
        val = f'{val}'
        table.add_row(arg, val)
    return s,table

In [ ]:
#export
@call_parse
def inspect_container(container_name:Param("The name of a downloaded DockerHub container", str)):
    "Reports all possible configuration values for a container"
    cmd = ['rekcod', container_name]
    res = _run(cmd)
    image_and_cmd, arguments = _format_inspection(res)
    console.print(image_and_cmd)
    console.print(arguments)

With this function, we can quickly see what all possible parameters we can setup for a docker container. This also has the shortcut of `di` (shorthand for Docker Investigate, how I envision this in my head).

We can see an example output below:

> **NOTE**: These do not include inherit arguments for general docker, such as `-v` to mount volumes, and so forth. For an exhaustive list of those, see the documentation [here](https://docs.docker.com/engine/reference/commandline/run/)

In [ ]:
#hide_input
img,arg = _format_inspection(base)
console.print(img)
console.print(arg)

Image: nbdev

Command for Running Container: docker run 

Possible Arguments:

┏━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃   Argument(s)    ┃ Value                                                                  ┃
┡━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│     --expose     │ 8888/tcp                                                               │
├──────────────────┼────────────────────────────────────────────────────────────────────────┤
│        -l        │ repo2docker.ref='None'                                                 │
├──────────────────┼────────────────────────────────────────────────────────────────────────┤
│        -l        │ repo2docker.repo='local'                                               │
├──────────────────┼────────────────────────────────────────────────────────────────────────┤
│        -l        │ repo2docker.version='2021.08.0'                                        │
├──────────────────┼────────────────────────────────────────────────────────────────────────┤
│        -e        │ 'PATH=/home/runner/.local/bin:/home/runner/.local/bin:/srv/conda/envs… │
├──────────────────┼────────────────────────────────────────────────────────────────────────┤
│        -e        │ 'DEBIAN_FRONTEND=noninteractive'                                       │
├──────────────────┼────────────────────────────────────────────────────────────────────────┤
│        -e        │ 'LC_ALL=en_US.UTF-8'                                                   │
├──────────────────┼────────────────────────────────────────────────────────────────────────┤
│        -e        │ 'LANG=en_US.UTF-8'                                                     │
├──────────────────┼────────────────────────────────────────────────────────────────────────┤
│        -e        │ 'LANGUAGE=en_US.UTF-8'                                                 │
├──────────────────┼────────────────────────────────────────────────────────────────────────┤
│        -e        │ 'SHELL=/bin/bash'                                                      │
├──────────────────┼────────────────────────────────────────────────────────────────────────┤
│        -e        │ 'USER=runner'                                                          │
├──────────────────┼────────────────────────────────────────────────────────────────────────┤
│        -e        │ 'HOME=/home/runner'                                                    │
├──────────────────┼────────────────────────────────────────────────────────────────────────┤
│        -e        │ 'APP_BASE=/srv'                                                        │
├──────────────────┼────────────────────────────────────────────────────────────────────────┤
│        -e        │ 'NPM_DIR=/srv/npm'                                                     │
├──────────────────┼────────────────────────────────────────────────────────────────────────┤
│        -e        │ 'NPM_CONFIG_GLOBALCONFIG=/srv/npm/npmrc'                               │
├──────────────────┼────────────────────────────────────────────────────────────────────────┤
│        -e        │ 'CONDA_DIR=/srv/conda'                                                 │
├──────────────────┼────────────────────────────────────────────────────────────────────────┤
│        -e        │ 'NB_PYTHON_PREFIX=/srv/conda/envs/notebook'                            │
├──────────────────┼────────────────────────────────────────────────────────────────────────┤
│        -e        │ 'NB_ENVIRONMENT_FILE=/tmp/env/environment.lock'                        │
├──────────────────┼────────────────────────────────────────────────────────────────────────┤
│        -e        │ 'KERNEL_PYTHON_PREFIX=/srv/conda/envs/notebook'                        │
├──────────────────┼────────────────────────────────────────────────────────────────────────┤
│        -e        │ 'REPO_DIR=/home/runner'                                                │
├──────────────────┼────────────────────────────────